In [1]:
import numpy as np

# Notebook compilation of data
An attempt to compile known GC data, and select the best/latest available.

## TODO
- [ ] Check Names, some alternate but same cluster
- [ ] Make sensible selection of best
- [ ] Check Gran Physical Coords
- [ ] Check Systematics of AMR  
    - [ ] Differences in Isochrones, BaSTi, DSED, BaSTi Alpha enhanced, BaSTI sometimes corrected BaSTi with an age shift due to atomic diffusion ect
    

## Name Preferencse
Alt Names gives preferences of  variants on a single name  
name_parse gives the preferences of alternative names of a GC


In [2]:
alt_name_parse = {f"Ter{i}": f"Terzan{i}" for i in np.arange(1, 15)}
alt_name_parse = alt_name_parse | {f"Palomar{i}": f"Pal{i}" for i in np.arange(1, 20)}
alt_name_parse = alt_name_parse | {f"Djor{i}": f"Djorg{i}" for i in np.arange(1, 4)}
alt_name_parse = alt_name_parse | {f"Dor{i}": f"Djorg{i}" for i in np.arange(1, 4)}
alt_name_parse["2MS-GC01"] = "GLIMPSE01"
alt_name_parse["2MASSGC01"] = "GLIMPSE01"
alt_name_parse["2MS-GC02"] = "GLIMPSE02"
alt_name_parse["2MASSGC02"] = "GLIMPSE02"
alt_name_parse["NGC0104"] = "NGC104"
alt_name_parse["NGC0288"] = "NGC288"
alt_name_parse["NGC0362"] = "NGC362"

alt_name_parse["ESO280SC06"] = "ESO-SC06"
alt_name_parse["ESO452SC11"] = "ESO452"

alt_name_parse["Ryu05"] = "Ryu059"
alt_name_parse["Ryu87"] = "Ryu879"


In [3]:
name_parse = {}
name_parse["1636-283"] = "ESO452"
name_parse["NGC104"] = "47Tuc"
name_parse["NGC1904"] = "M79"
name_parse["ESO371"] = "E3"
name_parse['Laevens1'] = "Crater"
name_parse["NGC4590"] = "M68"
name_parse["NGC5024"] = "M53"
name_parse["NGC5139"] = "oCen"
name_parse["NGC5272"] = "M3"
name_parse["NGC5904"] = "M5"
name_parse["Arp1"] = "Pal14"
name_parse["NGC6838"] = "M71"

name_parse["NGC7078"] = "M15"
name_parse["NGC6341"] = "M92"

name_parse["RLGC1"] = "Ryu059"
name_parse["RLGC2"] = "Ryu879"

name_parse["Ryu05"] = "Ryu059"
name_parse["Ryu87"] = "Ryu879"
name_parse["ESO280"] = "ESO-SC06"

# M71
# M15, M92
# RLGC1, RLGC2
# ESO280

## Funcs to Add data

In [4]:
paper_age = {}
data = {}


def data_add_function(data, new_data, data_name, check_GCs = ["ESO280"]):
    names = np.array(list(new_data.keys()), dtype="<U50")
    names = process_names(names)
    known_names = np.array(list(data.keys()), dtype="<U50")
    new_filt = np.isin(names,known_names,invert=True)
    if new_filt.sum()>0:
        N_new = new_filt.sum()
        print("Number New GCs:",N_new)
        print("New GCs:")
        print(names[new_filt])
        
    for name in names:
        gc_data = data.get(name, {})
        keys = list(new_data[name].keys())
        for k in keys:
            prop_data = gc_data.get(k, {})
            x = new_data[name][k]
            if x not in ['', '-']:
                prop_data[data_name] = x
            gc_data[k] = prop_data
        data[name] = gc_data
    gcs = np.array(list(data.keys()))
    Ngcs = len(gcs)
    print(f"Number of total GCs: {Ngcs}")
    check_filt = np.isin(gcs, check_GCs)
    if check_filt.sum() > 0:
        print("Check GCs present!")
        print(gcs[check_filt])

    return data

def process_names(names):
    known_names = list(name_parse.keys())
    new_names = np.empty_like(names)
    for i, n in enumerate(names):
        n = alt_name_parse.get(n, n)
        if n not in known_names:
            print(f"New GC: {n}")
            name_parse[n] = n
            print(name_parse[n])
        if n == '':
            print("Blank name error!")
        new_names[i] = name_parse[n]
    return new_names

def data_array_to_gc_dic(data_array, original_columns):
    columns = parse_columns(original_columns)
    name_column = (columns == 'name')
    known_names = list(name_parse.keys())
    Nrow = np.shape(data_array)[0]
    Nname_col = name_column.sum()
    '''Process Names'''
    if Nname_col == 0:
        print('Error, No names found')
    elif Nname_col == 1:
        names = data_array[:, name_column].reshape(-1)
        names = process_names(names)
    elif Nname_col > 1:
        alt_names = data_array[:, name_column].reshape(-1, Nname_col)
        names = np.empty((Nrow), dtype='<U50')
        for i in range(Nrow):
            gc_names = alt_names[i, :]
            in_filt = np.isin(gc_names, known_names) * (gc_names != '')
            if in_filt.sum() > 0:
                n = gc_names[in_filt][0]
                names[i] = name_parse[alt_name_parse.get(n, n)]
                for n in gc_names:
                    name_parse[n] = names[i]
            else:
                # print(f"New GC: {gc_names}")
                gc_nam = gc_names[gc_names != ''][0]
                gc_nam = alt_name_parse.get(gc_nam, gc_nam)
                for n in gc_names:
                    name_parse[n] = gc_nam
                names[i] = gc_nam
            if names[i] == '':
                print("Blank Error!")
                print(names[i])

    '''Process Data'''
    not_name = np.logical_not(name_column)
    props = columns[not_name]
    props_data = data_array[:, not_name]  # .astype(float)
    props_data[props_data == '-'] = 'NAN'
    props_data[props_data == ''] = 'NAN'
    props_data = props_data.astype(float)
    gc_dic = {}
    for ind_name, name in enumerate(names):
        gc_dic[name] = {}
        for (ind_p, p) in enumerate(props):
            x = props_data[ind_name, ind_p]
            if x != np.nan:
                if np.isfinite(x):
                    gc_dic[name][p] = x
        # gc_dic[name] = {p: props_data[ind_name, ind_p]
        #                 for (ind_p, p) in enumerate(props)}
    return gc_dic


def data_dic_to_gc_dic(data_dic):
    original_columns = list(data_dic.keys())
    Ncols = len(original_columns)
    Nrows = len(data_dic[original_columns[0]])
    data_array = np.empty((Nrows, Ncols), dtype="<U50")
    for n, col in enumerate(original_columns):
        data_array[:, n] = np.array(data_dic[col])
    return data_array_to_gc_dic(data_array, original_columns)

In [5]:
def create_col_parse_dic():
    labels = {}
    labels["name"] = ["Name", "Names", "name", "names", "cluster", "NGC", "ngc", "name2", "Name2"]
    labels["Fe_H"] = ["Fe_H", "Met", "Metallicity"]
    labels["Fe_H_err"] = ["Fe_H_err", "Met_err", "Metallicity_err",
                          "Fe_H_Err", "Met_Err", "Metallicity_Err"]
    labels["Age"] = ["Age"]
    labels["Age_err"] = ["Age_err", "Age_Err"]
    labels["Alpha_Fe"] = ["Alpha_Fe", "Alpha"]

    labels["Mv"] = ["Mv", "Mvt"]
    labels["m_Mv"] = ["m_Mv"]
    labels["m_Mv_Err"] = ["m_Mv_Err"]
    labels["Mass"] = ["Mass"]

    labels["dist"] = ["dsun", "dist"]
    parse_dic = {}
    for l in list(labels.keys()):
        for n in labels[l]:
            parse_dic[n] = l
    return parse_dic


col_parse_dic = create_col_parse_dic()


def parse_columns(original_columns):
    known_labels = np.array(list(col_parse_dic.keys()))
    unknown = np.isin(original_columns, known_labels, invert=True)
    if unknown.sum() > 0:
        print("Unknown labels:")
        print(np.array(original_columns)[unknown])
    parse_columns = np.array([col_parse_dic.get(c, c) for c in original_columns])
    return parse_columns

# Larger Data Files
Read in csv/txt files
## Dynamics

### Vasiliev

In [6]:
def load_Vasilev21_Data_Dic():
    # Eugene Vasilev read script
    tab = np.loadtxt('GC_Sky_Vasilev2021.txt', dtype=str)
    sky_data = {}
    sky_data['name'] = tab[:, 0]   # 0th column is the cluster name (string)
    sky_data['name2'] = tab[:, 1]   # 0th column is the cluster name (string)
    tab = tab[:, 2:].astype(float)  # remaining columns are numbers
    sky_data['ra_deg'] = tab[:, 0]   # coordinates of cluster centers [deg]
    sky_data['dec_deg'] = tab[:, 1]
    sky_data['dist'] = tab[:, 2]   # distance [kpc]
    diste = tab[:, 3]   # distance error [kpc]
    diste[~np.isfinite(diste)] = sky_data['dist'][~np.isfinite(diste)] * 0.05
    sky_data['dist_err'] = diste   # distance error [kpc]
    sky_data['vlos'] = tab[:, 4]   # line-of-sight velocity [km/s]
    vlose = tab[:, 5]   # its error estimate
    # assumed error of at least 2 km/s on line-of-sight velocity
    sky_data['vlos_err'] = np.maximum(vlose, 2.0)
    sky_data['pmra'] = tab[:, 8]   # mean proper motion [mas/yr]
    sky_data['pmdec'] = tab[:, 9]
    sky_data['pmra_err'] = tab[:, 10]   # its uncertainty
    sky_data['pmdec_err'] = tab[:, 11]
    # correlation coefficient for errors in two PM components
    sky_data['pmcorr'] = tab[:, 12]
    # assumed error of 0.1 mag in distance modulus when not known
    for (n1,n2) in zip(sky_data["name"], sky_data["name2"]):
        print(n1,n2)
    return sky_data


Vasilev21_data_dic = load_Vasilev21_Data_Dic()
Vasilev21_dic = data_dic_to_gc_dic(Vasilev21_data_dic)
data = data_add_function(data, Vasilev21_dic, data_name='Vasilev21')
paper_age["Vasilev21"] = 2021

NGC104 47Tuc
NGC288 -
NGC362 -
Whiting1 -
NGC1261 -
Pal1 -
AM1 -
Eridanus -
Pal2 -
NGC1851 -
NGC1904 M79
NGC2298 -
NGC2419 -
Ko2 -
Pyxis -
NGC2808 -
E3 -
Pal3 -
NGC3201 -
ESO93 -
Pal4 -
Crater -
Bliss1 -
Ko1 -
NGC4147 -
NGC4372 -
Rup106 -
NGC4590 M68
BH140 -
NGC4833 -
NGC5024 M53
NGC5053 -
Kim3 -
NGC5139 oCen
NGC5272 M3
NGC5286 -
AM4 -
NGC5466 -
NGC5634 -
NGC5694 -
IC4499 -
Munoz1 -
NGC5824 -
Pal5 -
NGC5897 -
NGC5904 M5
NGC5927 -
NGC5946 -
BH176 -
NGC5986 -
FSR1716 -
Pal14 -
Lynga7 BH184
NGC6093 M80
Ryu059 RLGC1
NGC6121 M4
NGC6101 -
NGC6144 -
NGC6139 -
Terzan3 -
NGC6171 M107
1636-283 ESO452
NGC6205 M13
NGC6229 -
NGC6218 M12
FSR1735 -
NGC6235 -
NGC6254 M10
NGC6256 -
Pal15 -
NGC6266 M62
NGC6273 M19
NGC6284 -
NGC6287 -
NGC6293 -
NGC6304 -
NGC6316 -
NGC6341 M92
NGC6325 -
NGC6333 M9
NGC6342 -
NGC6356 -
NGC6355 -
NGC6352 -
IC1257 -
Terzan2 HP3
NGC6366 -
Terzan4 HP4
HP1 BH229
FSR1758 -
NGC6362 -
Liller1 -
NGC6380 Ton1
Terzan1 HP2
Ton2 Pismis26
NGC6388 -
NGC6402 M14
NGC6401 -
NGC6397 -
VVVCL00

### Baumb Distances

In [7]:
def Baumgardt_GC_obs_load_sky():
    tab = np.loadtxt("Baumgardt21_sky.txt", dtype=str, skiprows=2)
    sky_data = {}
    Name = tab[:, 0].astype('<U20')   # 0th column is the cluster name (string)
    Name = np.char.replace(Name, ' ', '')
    Name = np.char.replace(Name, '_', '')
    Name = np.char.replace(Name, '-', '')
    Name = np.char.replace(Name, 'Ter', 'Terzan')
    Name = np.char.replace(Name, 'Lae', 'Laevens')
    Name = np.char.replace(Name, 'Djor', 'Djorg')
    Name = np.char.replace(Name, 'Lil', 'Liller')
    sky_data['name'] = Name
    tab = tab[:, 1:].astype(float)  # remaining columns are numbers
    sky_data['ra_deg'] = tab[:, 0]   # coordinates of cluster centers [deg]
    sky_data['dec_deg'] = tab[:, 1]
    sky_data['l_deg'] = tab[:, 2]   # coordinates of cluster centers [deg]
    sky_data['b_deg'] = tab[:, 3]   # coordinates of cluster centers [deg]
    sky_data['dist'] = tab[:, 4]   # distance [kpc]
    diste = tab[:, 5]   # distance error [kpc]
    diste[~np.isfinite(diste)] = sky_data['dist'][~np.isfinite(diste)] * 0.05
    sky_data['dist_err'] = diste   # distance error [kpc]
    sky_data['R_GC'] = tab[:, 6]   # distance error [kpc]
    sky_data['vlos'] = tab[:, 7]   # line-of-sight velocity [km/s]
    vlose = tab[:, 8]   # its error estimate
    # np.maximum(vlose, 2.0)  # assumed error of at least 2 km/s on line-of-sight velocity
    sky_data['vlos_err'] = vlose
    sky_data['pmra'] = tab[:, 9]   # mean proper motion [mas/yr]
    sky_data['pmra_err'] = tab[:, 10]   # its uncertainty
    sky_data['pmdec'] = tab[:, 11]
    sky_data['pmdec_err'] = tab[:, 12]
    # correlation coefficient for errors in two PM components
    sky_data['pmcorr'] = tab[:, 13]
    # assumed error of 0.1 mag in distance modulus when not known
    return sky_data


Baumgardt21_data_dic = Baumgardt_GC_obs_load_sky()
Baumgardt21_dic = data_dic_to_gc_dic(Baumgardt21_data_dic)
data = data_add_function(data, Baumgardt21_dic, data_name='Baumgardt21')
paper_age["Baumgardt21"] = 2021.5


Unknown labels:
['ra_deg' 'dec_deg' 'l_deg' 'b_deg' 'dist_err' 'R_GC' 'vlos' 'vlos_err'
 'pmra' 'pmra_err' 'pmdec' 'pmdec_err' 'pmcorr']
New GC: GLIMPSE01
GLIMPSE01
New GC: GLIMPSE02
GLIMPSE02
New GC: SagittariusII
SagittariusII
Number New GCs: 3
New GCs:
['GLIMPSE01' 'GLIMPSE02' 'SagittariusII']
Number of total GCs: 173


## AMR + Mass/Magnitude

### Baumgardt Cluster Mass
https://ui.adsabs.harvard.edu/abs/2018MNRAS.478.1520B/abstract  
https://people.smp.uq.edu.au/HolgerBaumgardt/globular/parameter.html

In [8]:
Baumgardt_GC_array = np.loadtxt("Baumgardt_GC_Mass.csv",
                                skiprows=1, dtype=str, delimiter=',')
Baumgardt_GC_Columns = ["Name", "Mass"]
print(np.shape(Baumgardt_GC_array))
Baumgardt_GC_dic = data_array_to_gc_dic(Baumgardt_GC_array, Baumgardt_GC_Columns)
data = data_add_function(data, Baumgardt_GC_dic, data_name='Baumgardt_Mass18')
paper_age["Baumgardt_Mass18"] = 2018

(161, 2)
New GC: 2MASS
2MASS
New GC: 2MASS
2MASS
Number New GCs: 1
New GCs:
['2MASS']
Number of total GCs: 174


### Kruijssen19 Sample
Data used in Kruijssen19. 3 Sources, Kuijssen takes average of available. Use all from sources, as some missing data added by other sources.

In [9]:
Kruijssen19_array = np.loadtxt("Kruijssen19.csv", skiprows=1, dtype=str, delimiter=',')
# Name, Met, Alpha, Age, Age Err
Kruijssen19_Columns = ['Name', 'Mv', 'Mass', 'Age', 'Age_Err', 'Fe_H']
Kruijssen19_array[:, 2] = (10 ** np.array(Kruijssen19_array[:, 2].astype(float))).astype(str)
Kruijssen19_dic = data_array_to_gc_dic(Kruijssen19_array, Kruijssen19_Columns)
data = data_add_function(data, Kruijssen19_dic, data_name='Kruijssen19')
paper_age["Kruijssen19"] = 2013

Number of total GCs: 174


#### Dotter
Dotter 11 added onto the back  
https://ui.adsabs.harvard.edu/abs/2010ApJ...708..698D/abstract  
https://ui.adsabs.harvard.edu/abs/2011ApJ...738...74D/abstract  

In [10]:
Dotter10_array = np.loadtxt("Dotter10.csv", skiprows=1, dtype=str, delimiter=',')
# Name, Met, Alpha, Age, Age Err
Dotter10_Columns = ['Name', 'Fe_H', 'Alpha_Fe', 'Age', 'Age_Err']
print(np.shape(Dotter10_array))
Dotter10_dic = data_array_to_gc_dic(Dotter10_array, Dotter10_Columns)
data = data_add_function(data, Dotter10_dic, data_name='Dotter10')
paper_age['Dotter10'] = 2010

(62, 5)
New GC: Ruprecht106
Ruprecht106
Number New GCs: 1
New GCs:
['Ruprecht106']
Number of total GCs: 175


#### Forbes+Bridges10
https://ui.adsabs.harvard.edu/abs/2010MNRAS.404.1203F/abstract  
Note No Age errors?

In [11]:
Forbes10_array = np.loadtxt("Forbes10.csv", skiprows=1, dtype=str, delimiter=',')
Forbes10_Columns = ['Name', 'Age', 'Met']
# No Age error?
print(np.shape(Forbes10_array))
Forbes10_dic = data_array_to_gc_dic(Forbes10_array, Forbes10_Columns)
data = data_add_function(data, Forbes10_dic, data_name='Forbes10')
paper_age["Forbes10"] = 2010

(88, 3)
New GC: Berk29
Berk29
New GC: Saurer1
Saurer1
Number New GCs: 2
New GCs:
['Berk29' 'Saurer1']
Number of total GCs: 177


#### VandenBerg13
https://ui.adsabs.harvard.edu/abs/2013ApJ...775..134V/abstract  
Following Kruijssen, +1Gyr Uncertainty

In [12]:
VandenBerg13_array = np.loadtxt("VandenBerg13.csv", skiprows=1, dtype=str, delimiter=',')
#NGC, Name, Met, Age, Age_Err, Mv
VandenBerg13_Columns = ['NGC', 'Name', 'Met', 'Age', 'Age_Err', 'Mv']
print(np.shape(VandenBerg13_array))
print("Adding 1GYR to errors")
VandenBerg13_array[:, 4] = (VandenBerg13_array[:, 4].astype(float) + 1).astype(str)
VandenBerg13_dic = data_array_to_gc_dic(VandenBerg13_array, VandenBerg13_Columns)
data = data_add_function(data, VandenBerg13_dic, data_name='VandenBerg13')
paper_age["VandenBerg13"] = 2013

(55, 6)
Adding 1GYR to errors
Number of total GCs: 177


### Harris10
https://physics.mcmaster.ca/~harris/mwgc.dat

In [13]:
Harris10_array = np.loadtxt("Harris10.csv", skiprows=1, dtype=str, delimiter=',')
#Name,Met, Mvt
Harris10_Columns = ['Name', 'Met', 'Mvt']
Harris10_dic = data_array_to_gc_dic(Harris10_array, Harris10_Columns)
data = data_add_function(data, Harris10_dic, data_name='Harris10')

paper_age["Harris10"] = 2010

Number of total GCs: 177


### Roediger13
https://ui.adsabs.harvard.edu/abs/2014ApJS..210...10R/abstract

In [14]:
Roediger13_array = np.loadtxt("Roediger13.csv", skiprows=1, dtype=str, delimiter=',')
Roediger13_Columns = ['Name', 'Age', 'Age_Err', 'Fe_H', 'Fe_H_err']
print(np.shape(Roediger13_array))
Roediger13_dic = data_array_to_gc_dic(Roediger13_array, Roediger13_Columns)
data = data_add_function(data, Roediger13_dic, data_name='Roediger13')
paper_age["Roediger13"] = 2013

(41, 5)
Number of total GCs: 177


### OMalley17
https://ui.adsabs.harvard.edu/abs/2017ApJ...838..162O/abstract

In [15]:
OMalley17_array = np.loadtxt("OMalley17.csv", skiprows=1, dtype=str, delimiter=',')
#Name, m_Mv, Age, Age_Err, Fe_H
OMalley17_Columns = ["Name", "m_Mv", "m_Mv_Err", "Age", "Age_Err", "Fe_H"]
print(np.shape(OMalley17_array))
OMalley17_dic = data_array_to_gc_dic(OMalley17_array, OMalley17_Columns)
data = data_add_function(data, OMalley17_dic, data_name='OMalley17')

paper_age["OMalley17"] = 2017

(22, 6)
Number of total GCs: 177


### Vasquez18
https://ui.adsabs.harvard.edu/abs/2018A%26A...619A..13V/abstract  
Use  the most recent Fe_H_D16 data?  
Also contains radial velocities!

In [16]:
Vasquez18_array = np.loadtxt("Vasquez18.csv", skiprows=1, dtype=str, delimiter=',')
# Name, Fe_H D16, Fe_H_err, Fe_H_S12, Fe_H_err, Fe_H_V15, Fe_H_err, Fe_H_C09
Vasquez18_array = Vasquez18_array[:, :3]
Vasquez18_Columns = ["Name", "Fe_H", "Fe_H_err"]
print(np.shape(Vasquez18_array))
Vasquez18_dic = data_array_to_gc_dic(Vasquez18_array, Vasquez18_Columns)
data = data_add_function(data, Vasquez18_dic, data_name='Vasquez18')

paper_age["Vasquez18"] = 2018

(48, 3)
Number of total GCs: 177


### Saviane12
https://ui.adsabs.harvard.edu/abs/2012A%26A...540A..27S/abstract

Use Fe_H_C09 data?

In [17]:
Saviane12_array = np.loadtxt("Saviane12.csv", skiprows=1, dtype=str, delimiter=',')
#Name, Name2, W, sigmaW, Fe_H_C09, err, Fe_H_H10, H10wt, Fe_H_C09_AP1, e
Saviane12_array = Saviane12_array[:, [0, 1, 4, 5]]
print(np.shape(Saviane12_array))
Saviane12_Columns = ["Name", "Name2", "Fe_H", "Fe_H_err"]
Saviane12_dic = data_array_to_gc_dic(Saviane12_array, Saviane12_Columns)
data = data_add_function(data, Saviane12_dic, data_name='Saviane12')

paper_age["Saviane12"] = 2012

(71, 4)
Number of total GCs: 177


### Oliveria20
https://ui.adsabs.harvard.edu/abs/2020ApJ...891...37O/abstract  
HST


In [18]:
Oliveria20_array = np.loadtxt("Oliveria20.csv", skiprows=1, dtype=str, delimiter=',')
Oliveria20_Columns = ["Name", "Age", "Age_Err_1os",
                      "Age_Err_1us", "Fe_H", "Fe_H_Err_1os", "Fe_H_Err_1us"]
Oliveria20_array[:, 2] = (10 ** np.array(Oliveria20_array[:, 2].astype(float))).astype(str)
Oliveria20_dic = data_array_to_gc_dic(Oliveria20_array, Oliveria20_Columns)
data = data_add_function(data, Oliveria20_dic, data_name='Oliveria20')
paper_age["Oliveria20"] = 2020

Unknown labels:
['Age_Err_1os' 'Age_Err_1us' 'Fe_H_Err_1os' 'Fe_H_Err_1us']
Number of total GCs: 177


### CAPOS21
https://ui.adsabs.harvard.edu/abs/2021A%26A...652A.157G/abstract


In [19]:
CAPOS21_array = np.loadtxt("CAPOS21.csv", skiprows=1, dtype=str, delimiter=',')
CAPOS21_Columns = ["Name", "Fe_H", "Fe_H_Err", "Alpha_H", "Alpha_H_Err"]
CAPOS21_array[:, 2] = (10 ** np.array(CAPOS21_array[:, 2].astype(float))).astype(str)
CAPOS21_dic = data_array_to_gc_dic(CAPOS21_array, CAPOS21_Columns)
data = data_add_function(data, CAPOS21_dic, data_name='CAPOS21')
paper_age["CAPOS21"] = 2020

Unknown labels:
['Alpha_H' 'Alpha_H_Err']
Number of total GCs: 177


### GRAN22
Gran22: https://ui.adsabs.harvard.edu/abs/2022MNRAS.509.4962G/abstract  
Errors?  
Correct form of PMs?

In [20]:

Gran22_array = np.loadtxt("Gran22.csv", skiprows=1, dtype=str, delimiter=',')
Gran22_Columns = ["Name", "l_deg", "b_deg", "ra_deg", "dec_deg", "mua_cos", "mud",
                  "mul_cosb", "mub", "Fe_H", "Fe_H_Err", "Mass", "Mass_Err", "Dist", "Mv", "vlos", "vlos_Err"]

Gran22_array[:, 2] = (10 ** np.array(Gran22_array[:, 2].astype(float))).astype(str)
Gran22_dic = data_array_to_gc_dic(Gran22_array, Gran22_Columns)
data = data_add_function(data, Gran22_dic, data_name='Gran22')
paper_age["Gran22"] = 2020

Unknown labels:
['l_deg' 'b_deg' 'ra_deg' 'dec_deg' 'mua_cos' 'mud' 'mul_cosb' 'mub'
 'Mass_Err' 'Dist' 'vlos' 'vlos_Err']
New GC: Gran2
Gran2
New GC: Gran3
Gran3
New GC: Gran4
Gran4
New GC: Gran5
Gran5
Number New GCs: 4
New GCs:
['Gran2' 'Gran3' 'Gran4' 'Gran5']
Number of total GCs: 181


### Valenti09
Colour magnitude  
Paper: https://ui.adsabs.harvard.edu/abs/2010MNRAS.402.1729V/abstract  
Data: http://www.bo.astro.it/~GC/ir_archive/  
Compilation of Previous Valenti data - should be updated reguarly?

In [21]:
Valenti09_array = np.loadtxt("Valenti09.csv", skiprows=1, dtype=str, delimiter=',')
Valenti09_Columns = ["Name", "l_deg", "b_deg", "ra_deg", "dec_deg", "mua_cos", "mud",
                     "mul_cosb", "mub", "Fe_H", "Fe_H_Err", "Mass", "Mass_Err", "Dist",
                     "Mv", "vlos", "vlos_Err"]


Valenti09_Columns = ["Name", "l_deg", "b_deg", "dsun", "RGC", "X", "Y", "Z", "E(B-V)", "(m-M)0", "Fe_H",
                     "[M/H]", "JRC", "HRC", "KRC", "JBump", "HBump", "KBump", "JTip", "HTip", "KTip",
                     "MBolBump", "MBolTip"]


Valenti09_array[:, 2] = (10 ** np.array(Valenti09_array[:, 2].astype(float))).astype(str)
Valenti09_dic = data_array_to_gc_dic(Valenti09_array, Valenti09_Columns)
data = data_add_function(data, Valenti09_dic, data_name='Valenti09')
paper_age["Valenti09"] = 2009

Unknown labels:
['l_deg' 'b_deg' 'RGC' 'X' 'Y' 'Z' 'E(B-V)' '(m-M)0' '[M/H]' 'JRC' 'HRC'
 'KRC' 'JBump' 'HBump' 'KBump' 'JTip' 'HTip' 'KTip' 'MBolBump' 'MBolTip']
New GC: Dorg1
Dorg1
New GC: Dorg2
Dorg2
Number New GCs: 2
New GCs:
['Dorg1' 'Dorg2']
Number of total GCs: 183


### Bica16
https://ui.adsabs.harvard.edu/abs/2016PASA...33...28B/abstract
Candidate GCs with sky, Metallicity and further Chem data.  
Not clear which are actually GCs or not


# Individual_Data
Individual papers and sources

In [22]:
def add_individual_data(data, individual_data):
    papers = list(individual_data.keys())
    for p in papers:
        print(p)
        original_new_data =individual_data[p]
        original_names = np.array(list(original_new_data.keys()))
        names = process_names(original_names)
        try:
            new_data = {new_n:original_new_data[old_n] for (new_n, old_n) in zip(names, original_names)}
            data = data_add_function(data, new_data, data_name=p)
        except Exception as e:
            print(e)
            print(names)
            print(original_names)
            print(original_new_data.keys())

    return data

In [23]:
individual_data = {}

## Small Papers

### Kerber18
NGC6522, NGC6626(M28), NGC6362  
Kerber18: https://ui.adsabs.harvard.edu/abs/2018ApJ...853...15K/abstract  
Alpha enhanced BASTI or DSED sochrone fitting. Use DSED to avoid attomic difusion problems.  
Take the smallest ChiSq.

In [24]:
individual_data["Kerber18"] = {
    "NGC6522": {
        "Age": 11.4, "Age_Err": 1.0,
        "Fe_H": -1.15,
        # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
        "dist": 7.05, "dist_err": 0.16
    },
    "NGC6626": {
        "Age": 11.1, "Age_Err": 0.9,
        "Fe_H": -1.3,
        # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
        "dist": 5.18, "dist_err": 0.14
    },
    "NGC6362": {
        "Age": 12.8, "Age_Err": 1.0,
        "Fe_H": -1.15,
        # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
        "dist": 7.73, "dist_err": 0.18
    }
}

paper_age["Kerber18"] = 2018

### Dias15
NGC6528, NGC6553, M71, NGC6558, NGC6426, Terzan8
Dias15: https://ui.adsabs.harvard.edu/abs/2015A%26A...573A..13D/abstract  
Fe_H, Alpha_Fe, Mg, vlos

In [25]:
individual_data["Dias15"] = {
    "NGC6528": {
        "Fe_H": -0.13, "Fe_H_Err": 0.05,
        "vlos": 185, "vlos_err": 10,
        "Mg_Fe": 0.05, "Mg_Fe_Err": 0.09,
        "Alpha_Fe": 0.26, "Alpha_Fe_Err": 0.05,
    },
    "NGC6553": {
        "Fe_H": -0.133, "Fe_H_Err": 0.017,
        "vlos": 6, "vlos_err": 8,
        "Mg_Fe": 0.107, "Mg_Fe_Err": 0.009,
        "Alpha_Fe": 0.302, "Alpha_Fe_Err": 0.025,
    },
    "M71": {
        "Fe_H": -0.63, "Fe_H_Err": 0.05,
        "vlos": -42, "vlos_err": 18,
        "Mg_Fe": 0.25, "Mg_Fe_Err": 0.07,
        "Alpha_Fe": 0.293, "Alpha_Fe_Err": 0.032,
    },
    "NGC6558": {
        "Fe_H": -1.012, "Fe_H_Err": 0.013,
        "vlos": -210, "vlos_err": 16,
        "Mg_Fe": 0.26, "Mg_Fe_Err": 0.06,
        "Alpha_Fe": 0.23, "Alpha_Fe_Err": 0.06,
    },
    "NGC6426": {
        "Fe_H": -2.39, "Fe_H_Err": 0.11,
        "vlos": -242, "vlos_err": 11,
        "Mg_Fe": 0.38, "Mg_Fe_Err": 0.06,
        "Alpha_Fe": 0.24, "Alpha_Fe_Err": 0.05,
    },
    "Terzan8": {
        "Fe_H": -2.06, "Fe_H_Err": 0.17,
        "vlos": 135, "vlos_err": 19,
        "Mg_Fe": 0.41, "Mg_Fe_Err": 0.04,
        "Alpha_Fe": 0.21, "Alpha_Fe_Err": 0.04,
    }
}

paper_age["Dias15"] = 2015

### VandenBerg16
https://ui.adsabs.harvard.edu/abs/2016ApJ...827....2V/abstract  
M3, M15, M92. More information, but not very clear?

In [26]:
individual_data["Vandenberg16"] = {
    "M3": {
        "Age": 12.5
    },
    "M15": {
        "Age": 12.5
    },
    "M92": {
        "Age": 12.5
    },
}

paper_age["Vandenberg16"] = 2016

## Individual Clusters

### Pal2
https://ui.adsabs.harvard.edu/abs/2020MNRAS.493.2688B/abstract

In [27]:
individual_data["Bonatto20"] = {"Pal2": {'Age': 13.25, 'Age_Err': 0.12, "Fe_H": -1.58, "Fe_H_Err": 0.08,
                                         "Mass": 1.4e5, "Mass_Err": 0.4e5, "Mvt": -7.8}}
paper_age["Bonatto20"] = 2020

### Crater/Lavens1
Crater  
https://ui.adsabs.harvard.edu/abs/2016ApJ...822...32W/abstract

In [28]:

individual_data["Weisz16"] = {"Crater": {'Age': 7.5, 'Age_Err': 0.4,
                                         "Fe_H": -1.66, "Fe_H_Err": 0.04,
                                         "Mass": 9.9e3, "Mvt": -5.3}}
paper_age["Weisz16"] = 2016

### Bliss1
https://ui.adsabs.harvard.edu/abs/2019ApJ...875..154M/abstract

In [29]:
individual_data["Hempel14"] = {"Bliss1": {'Age': 9.2, 'Age_Err_1os': 1.6, "Age_Err_1us": -0.8,
                                          "Fe_H": -1.4,
                                          "Mass": 143, "Mass_Err": 37,
                                          "Mvt": 0, "Mvt_Err_1os": 1.7, "Mvt_Err_1us": -0.7}}
paper_age["Hempel14"] = 2014

### Kim3
https://ui.adsabs.harvard.edu/abs/2016ApJ...820..119K/abstract

In [30]:
individual_data["Kim16"] = {"Kim3": {'Age': 9.5, 'Age_Err_1os': 3, "Age_Err_1us": -1.6,
                                     "Fe_H": -1.6, "Fe_H_1os": 0.45, "Fe_H_1us": -0.3,
                                     "Mvt": 0.7, "Mvt_Err": 0.3}}
paper_age["Kim16"] = 2016

### Munoz1
https://ui.adsabs.harvard.edu/abs/2012ApJ...753L..15M/abstract

In [31]:
individual_data["Munoz12"] = {"Munoz1": {'Age': 12.5,
                                         "Fe_H": -1.5,
                                         "Mvt": -0.4, "Mvt_Err": 0.9}}

paper_age["Kim16"] = 2016

### BH176
https://ui.adsabs.harvard.edu/abs/2011A%26A...528A..70D/abstract

In [32]:
individual_data["Davoust11"] = {"BH176": {'Age': 6.5, "Age_Err": 0.5,
                                          "Fe_H": -0.1, "Fe_H_Err": 0.1,
                                          "Mvt": -3.82}}

paper_age["Davoust11"] = 2011

### FSR1716
https://ui.adsabs.harvard.edu/abs/2017A%26A...605A.128K/abstract

Age given as 10-12 in:
https://ui.adsabs.harvard.edu/abs/2016arXiv161103753B/abstract

In [33]:
individual_data["Andreas17"] = {"FSR1716": {  # 'Age': ?,
    "Fe_H": -1.38, "Fe_H_Err": 0.2,
    "Mass": 1.4e4, "Mass_Err_1os": 1.2e4, "Mass_Err_1us": 0.8e4,
    "Mvt": -5.1, "Mvt_Err": 1}}

paper_age["Andreas17"] = 2017

individual_data["Buckner16"] = {"FSR1716": {'Age': 11, "Age_Err": 1}}
paper_age["Buckner16"] = 2016

### Ryu 1 and 2
https://ui.adsabs.harvard.edu/abs/2018ApJ...863L..38R/abstract  
Age assumed 12.6, as an average of metal poor GCs. Could include it and have very wide errors?

In [34]:
individual_data["Ryu18"] = {
    "Ryu059": {  # 'Age': 12.6,
        "Fe_H": -2.2, "Fe_H_Err": 0.2, },
    "Ryu879": {  # 'Age': 12.6,
        "Fe_H": -2.1, "Fe_H_Err": 0.3, }
}

paper_age["Ryu18"] = 2018

### NGC6139
https://ui.adsabs.harvard.edu/abs/2015A%26A...583A..69B/abstract
errors of +- 0.015 +- 0.058  
Combine in Quadrature

In [35]:
individual_data["Bragaglia15"] = {"NGC6139": {
    "Fe_H": -1.579, "Fe_H_Err": np.sqrt((0.015**2) + (0.058**2)),
    "Mvt": -3.82}}

paper_age["Davoust11"] = 2011

### ESO452
Simpson 17
https://ui.adsabs.harvard.edu/abs/2017MNRAS.472.2856S/abstract


Cornish 06. 
Fits multiple isochrones, find a large range of metallicity and ages.
https://ui.adsabs.harvard.edu/abs/2006AJ....131.2543C/abstract

In [36]:
individual_data["Simpson17"] = {"ESO452": {
    "Fe_H": -0.81, "Fe_H_Err": 0.13,
    "Mass": 6.8e3, "Mass_Err": 3.4e3
}
}

paper_age["Simpson17"] = 2017

individual_data["Cornish06"] = {"ESO452": {"Age": 12.5, "Age_Err": 3.5,
                                           }
                                }
paper_age["Cornish06"] = 2006

### NGC6229
Johnson17: https://ui.adsabs.harvard.edu/abs/2017AJ....154..155J/abstract

Arlleno15: https://ui.adsabs.harvard.edu/abs/2015MNRAS.452..727A/abstract  
Gives age as consistent with 12+-1


In [37]:
individual_data["Johnson17"] = {"NGC6229": {
    "Fe_H": -1.13, "Fe_H_Err": 0.06,
}}

paper_age["Johnson17"] = 2017

individual_data["Arellano15"] = {"NGC6229": {"Age": 12, "Age_Err": 1,
                                             "Fe_H": -1.31, "Fe_H_Err": 0.12,
                                             }}
paper_age["Arellano15"] = 2015

### NGC6256
Cardelano20: https://ui.adsabs.harvard.edu/abs/2020ApJ...895...54C/abstract  
Isochrone fitting using Vasq18 Metallicity


In [38]:
individual_data["Cardelano20"] = {"NGC6256": {
    "Age": 13, "Age_Err": 0.5,
    "Fe_H": -1.62
}}

paper_age["Cardelano20"] = 2020

### HP1
Kerber20: https://ui.adsabs.harvard.edu/abs/2019MNRAS.484.5530K/abstract

Barbuy16: https://ui.adsabs.harvard.edu/abs/2016A%26A...591A..53B/abstract

In [39]:
individual_data["Kerber20"] = {"HP1": {
    "Age": 12.75, "Age_Err_1os": 0.86, "Age_Err_1us": -0.81,
    "Fe_H": -1.09, "Fe_H_Err_1os": 0.07, "Fe_H_Err_1us": -0.09,
    "Alpha_Fe": 0.4
}}

paper_age["Kerber20"] = 2020

individual_data["Barbuy16"] = {"HP1": {
    "Fe_H": -1.06, "Fe_H_Err": 0.1,
}}

paper_age["Barbuy16"] = 2016

### FSR1758
Romero21: https://ui.adsabs.harvard.edu/abs/2021A%26A...652A.158R/abstract  
Fits Isochrone, CAPOS APOGEE 

Vilanova19: https://ui.adsabs.harvard.edu/abs/2019ApJ...882..174V/abstract  
High Res Spectra. Prefered?

Barba19: https://ui.adsabs.harvard.edu/abs/2019ApJ...870L..24B/abstract

In [40]:
individual_data["Romero21"] = {"FSR1758": {
    "Age": 11.6, "Age_Err_1os": 1.25, "Age_Err_1us": -1.31,
    "Fe_H": -1.36, "Fe_H_Err": 0.05
}}

paper_age["Romero21"] = 2021

individual_data["Vilanova19"] = {"FSR1758": {
    "Fe_H": -1.58, "Fe_H_Err": 0.03,
    "Alpha_Fe": 0.32, "Alpha_Fe_Err": 0.01
}}

paper_age["Vilanova19"] = 2019

individual_data["Barba19"] = {"FSR1758": {
    "Fe_H": -1.5, "Fe_H": 0.3
}}

paper_age["Barba19"] = 2019

### NGC6402
DAntona22: https://ui.adsabs.harvard.edu/abs/2022ApJ...925..192D/abstract  
Johnson19: https://ui.adsabs.harvard.edu/abs/2019MNRAS.485.4311J/abstract

In [41]:
individual_data["DAntona22"] = {"NGC6402": {
    "Age": 12.5, "Age_Err": 0.75,
}}

paper_age["Johnson19"] = 2019
individual_data["Johnon19"] = {"NGC6402": {
    "Fe_H": -1.13, "Fe_H_Err": 0.05,
    "Alpha_Fe": 0.3
}}

paper_age["Johnson19"] = 2019

### Palomar 6
Souza21: https://ui.adsabs.harvard.edu/abs/2021A%26A...656A..78S/abstract

In [42]:
individual_data["Souza21"] = {"Pal6": {
    "Age": 12.4, "Age_Err": 0.9,
    "Fe_H": -1.10, "Fe_H_Err": 0.09,
    "Mvt": -6.79
}}

paper_age["Souza21"] = 2021

### NGC6440
Pallanca21: https://ui.adsabs.harvard.edu/abs/2021ApJ...913..137P/abstract

Munoz17: https://ui.adsabs.harvard.edu/abs/2017A%26A...605A..12M/abstract  
Na-0 anticorrelation  

In [43]:
individual_data["Pallanca21"] = {"NGC6440": {
    "Age": 13, "Age_Err": 1.5,
    "Fe_H": -1.34, "Fe_H_Err": 0.36,
    "Mvt": 1.12, "Mvt_Err": 0.12
}}

paper_age["Pallanca21"] = 2021

individual_data["Munoz17"] = {"NGC6440": {
    "Fe_H": -0.5, "Fe_H_Err": 0.03,
    "Age": 13
}}

paper_age["Munoz17"] = 2017

### UKS1
Fernandez20: https://ui.adsabs.harvard.edu/abs/2020A%26A...643A.145F/abstract  
Spectra of 6 stars, fit isochrone for age

In [44]:
individual_data["Fernandez20"] = {"UKS1": {
    "Age": 13.1, "Age_Err_1os": 0.93, "Age_Err_1us": -1.29,
    "Fe_H": -0.98, "Fe_H_Err": 0.11,
}}

paper_age["Fernandez20"] = 2020

### VVVCL001
Fernandez21: https://ui.adsabs.harvard.edu/abs/2021ApJ...908L..42F/abstract  
High resolution abundance analysis + isochrone fitting  
Alpha Available


In [45]:
individual_data["Fernandez21"] = {"VVVCL001": {
    "Age": 11.9, "Age_Err_1os": 3.12, "Age_Err_1us": -4.05,
    "Fe_H": -2.45, "Fe_H_Err": 0.24,
    # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
    "dist0": 8.22, "dist_err_1os": 1.84, "dist_err_1us": -1.93
}}

paper_age["Fernandez21"] = 2020

### Djorg2
Ortolani19: https://ui.adsabs.harvard.edu/abs/2019A%26A...627A.145O/abstract   
Isochrone fitting, BaSTi uncorrected?

In [46]:
individual_data["Ortolani19"] = {"Djorg2": {
    "Age": 12.7, "Age_Err_1os": 0.72, "Age_Err_1us": -0.69,
    "Fe_H": -1.11, "Fe_H_Err": 0.03,
    # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
    "dist": 8.75, "dist_err": 0.12
}}

paper_age["Ortolani19"] = 2019

### NGC6522
Barbuy21: https://ui.adsabs.harvard.edu/abs/2021A%26A...654A..29B/abstract
Spectra, Flames GIRAFEE  
Alpha available

In [47]:
individual_data["Barbuy21"] = {"NGC6522": {
    "Age": 12.7, "Age_Err_1os": 0.72, "Age_Err_1us": -0.69,
    "Fe_H": -1.05, "Fe_H_Err": 0.20,
    # "Fe_H":-2.37, "Fe_H_Err_1os":0.19,"Fe_H_Err_1us":-0.14,
    "dist": 8.75, "dist_err": 0.12,
    "vlos": -15.62, "vlos_err": 7.7
}}

paper_age["Barbuy21"] = 2021

### NGC6528
Munoz18: https://ui.adsabs.harvard.edu/abs/2018A%26A...620A..96M/abstract   
Alpha available too.

In [48]:
individual_data["Munoz18"] = {"NGC6528": {
    "Fe_H": -0.14, "Fe_H_Err": 0.03,
}}

paper_age["Munoz18"] = 2018

### NGC6544
Gran21: https://ui.adsabs.harvard.edu/abs/2021MNRAS.504.3494G/abstract

In [49]:
individual_data["Gran21"] = {"NGC6544": {
    "Fe_H": -1.44, "Fe_H_Err": 0.04,
    "Alpha_Fe": 0.2, "Alpha_Fe_Err": 0.04
}}

paper_age["Gran21"] = 2021

### ESO280
Simpson18:  https://ui.adsabs.harvard.edu/abs/2018MNRAS.477.4565S/abstract   
High Res Spectra

In [50]:
individual_data["Simpson18"] = {"ESO-SC06": {
    "Fe_H": -2.48, "Fe_H_Err_u1s": -0.11, "Fe_H_Err_o1s": 0.06,
    "Mass": 12e3, "Mass_Err": 2e3,
    "Alpha_Fe": 0.2, "Alpha_Fe_Err": 0.04,
    "dist": 15.2, "dist_err": 2.1,
    "vlos": 92.5, "vlos_err_o1s": 2.4, "vlos_err_u1s": -1.6
}}

paper_age["Simpson18"] = 2018

### NGC6553
Munoz20: https://ui.adsabs.harvard.edu/abs/2020MNRAS.492.3742M/abstract
High Res Spectra

In [51]:
individual_data["Munoz20"] = {"NGC6553": {
    "Fe_H": -0.14, "Fe_H_Err": 0.07,
    "Alpha_Fe": 0.11, "Alpha_Fe_Err": 0.05
}}

paper_age["Munoz20"] = 2020

### NGC6558
Barbuy18: https://ui.adsabs.harvard.edu/abs/2018A%26A...619A.178B/abstract
High Res Spectra  
Alpha Available

In [52]:
individual_data["Barbuy18"] = {"NGC6558": {
    "Fe_H": -1.17, "Fe_H_Err": 0.10,
}}

paper_age["Barbuy18"] = 2018

### Valenti11
Valenti11: https://ui.adsabs.harvard.edu/abs/2011MNRAS.414.2690V/abstract
High res infrared spectra

In [53]:
individual_data["Valenti11"] = {
    "NGC6624": {"Fe_H": -0.69, "Fe_H_Err": 0.02,
                "Alpha_Fe": 0.39, "Alpha_Fe_Err": 0.02,
                "vlos": 51, "vlos_Err": 3
                },
    "NGC6569": {"Fe_H": -0.79, "Fe_H_Err": 0.02,
                "Alpha_Fe": 0.43, "Alpha_Fe_Err": 0.02,
                "vlos": 47, "vlos_Err": 4
                }
}

paper_age["Valenti11"] = 2011

### NGC6569
Saracino19: https://ui.adsabs.harvard.edu/abs/2019ApJ...874...86S/abstract
Age from averaged isochorne from DSED, VR, and BaSTI

In [54]:
individual_data["Saracino19"] = {"NGC6569": {
    "dist": 10.1, "dist_err": 0.2,
    "Age": 12.8, "Age_Err": 1,
    "Fe_H": -1.17, "Fe_H_Err": 0.10,
}}

paper_age["Saracino19"] = 2019

### NGC6642
Balbinot10: https://ui.adsabs.harvard.edu/abs/2010IAUS..266..357B/abstract

In [55]:
individual_data["Balbinot10"] = {"NGC6642": {
    "Age": 10 * 10.14, "Age_Err_o1s": ((10**1.19) - (10**1.14)), "Age_Err_u1s": ((10**1.14) - (10**1.05)),
    "Fe_H": -1.80, "Fe_H_Err": 0.2,
    "dist": 8.05, "dist_err": 0.66,
}}

paper_age["Balbinot10"] = 2010

### NGC6809
Rain19: https://ui.adsabs.harvard.edu/abs/2019MNRAS.483.1674R/abstract  
Spectra, Chemical 

Diakogiannis14: https://ui.adsabs.harvard.edu/abs/2014MNRAS.437.3172D/abstract  
Dynamical Mass  

In [56]:
individual_data["Rain19"] = {"NGC6809": {
    "Fe_H": -2.01, "Fe_H_Err": 0.02,
    "Alpha_Fe": 0.4, "Alpha_Fe_Err": 0.04,
}}

paper_age["Rain19"] = 2019

individual_data["Diakogiannis14"] = {"NGC6809": {
    "Mass": 6.1e4, "Mass_Err_o1s": 0.51e4, "Mass_Err_u1s": -0.88e4,
}}

paper_age["Diakogiannis14"] = 2014

### Palomar 11
Lewis06: https://ui.adsabs.harvard.edu/abs/2006AJ....131.2538L/abstract

In [57]:
individual_data["Lewis06"] = {"Pal11": {
    "Age": 11.4, "Age_Err": 0.5,
    "dist": 14.3, "dist_err": 0.4,
}}

paper_age["Lewis06"] = 2006

### Segue3
Ortolani13: https://ui.adsabs.harvard.edu/abs/2013MNRAS.433.1966O/abstract

Hughes17: https://ui.adsabs.harvard.edu/abs/2017AJ....154...57H/abstract  
Dartmouth Geneva Models. Very, very young!

In [58]:
individual_data["Ortolani13"] = {"Segue3": {
    "Age": 3.2,
    "Fe_H": -0.8,
}}

paper_age["Ortolani13"] = 2013

individual_data["Hughes17"] = {"Segue3": {
    "Fe_H": -0.55, "Fe_H_Err_u1s": -0.12, "Fe_H_Err_o1s": 0.15,
    "Age": 2.6, "Age_Err": 0.4,
    "dist": 14.3, "dist_err": 0.4,
}}

paper_age["Hughes17"] = 2017

### Laevens3
Longeard19: https://ui.adsabs.harvard.edu/abs/2019MNRAS.490.1498L/abstract

In [59]:
individual_data["Longeard19"] = {"Laevens3": {
    "Age": 13.0, "Age_Err": 1.0,
    "dist": 61.4, "dist_err": 1.0,
    "Fe_H": -1.8, "Fe_H_Err": 0.1,
    "vlos": -70.2, "vlos_err": 0.5
}}

paper_age["Longeard19"] = 2019

### Palomar 11
Lewis06: https://ui.adsabs.harvard.edu/abs/2006AJ....131.2538L/abstract

In [60]:
individual_data["Lewis06"] = {"Pal11": {
    "Age": 11.4, "Age_Err": 0.5,
    "dist": 14.3, "dist_err": 0.4,
}}

paper_age["Lewis06"] = 2006

### NGC4833
Carretta14:  https://ui.adsabs.harvard.edu/abs/2014A%26A...564A..60C/abstract

In [61]:
individual_data["Carretta14"] = {"NGC4833": {
    "Age": 11.4, "Age_Err": 0.5,
    "dist": 14.3, "dist_err": 0.4,
}}

paper_age["Lewis06"] = 2006

### NGC6388
Carretta21: https://ui.adsabs.harvard.edu/abs/2021arXiv211112721C/abstract  
Metallicity only

In [62]:
# individual_data["Carretta21"] = {"NGC6388": {
#     "Fe_H":-0.480,
#     "Fe_H":-0.488,
# }}

# paper_age["Caretta21"] = 2021

### Terzan8
Carretta14: https://ui.adsabs.harvard.edu/abs/2014A%26A...561A..87C/abstract

In [63]:
individual_data["Carretta14"] = {"Terzan8":
                                 {"Fe_H": -2.27, "Fe_H_Err": 0.03}
                                 }

paper_age["Caretta14"] = 2014

### Liller1
Ferraro21: https://ui.adsabs.harvard.edu/abs/2021NatAs...5..311F/abstract  
Multiple populations, we use the oldest. Not clear if it is a genuine GC?

In [64]:
individual_data["Ferraro21"] = {"Liller1":
                                {"Fe_H": -0.4, "Alpha_Fe": 0.2,
                                 "Age": 12, "Age_Err": 1.5,
                                 }
                                }

paper_age["Ferraro21"] = 2021

## Add Indiviudal Data

In [65]:
data = add_individual_data(data, individual_data)

Kerber18
Number of total GCs: 183
Dias15
Number of total GCs: 183
Vandenberg16
Number of total GCs: 183
Bonatto20
Number of total GCs: 183
Weisz16
Number of total GCs: 183
Hempel14
Number of total GCs: 183
Kim16
Number of total GCs: 183
Munoz12
Number of total GCs: 183
Davoust11
Number of total GCs: 183
Andreas17
Number of total GCs: 183
Buckner16
Number of total GCs: 183
Ryu18
Number of total GCs: 183
Bragaglia15
Number of total GCs: 183
Simpson17
Number of total GCs: 183
Cornish06
Number of total GCs: 183
Johnson17
Number of total GCs: 183
Arellano15
Number of total GCs: 183
Cardelano20
Number of total GCs: 183
Kerber20
Number of total GCs: 183
Barbuy16
Number of total GCs: 183
Romero21
Number of total GCs: 183
Vilanova19
Number of total GCs: 183
Barba19
Number of total GCs: 183
DAntona22
Number of total GCs: 183
Johnon19
Number of total GCs: 183
Souza21
Number of total GCs: 183
Pallanca21
Number of total GCs: 183
Munoz17
Number of total GCs: 183
Fernandez20
Number of total GCs: 183


In [66]:
reverse_alt_name_dic = {}
for n in name_parse.keys():
    name = name_parse[n]
    alt_names = reverse_alt_name_dic.get(name, [])
    alt_names.append(n)
    reverse_alt_name_dic[name] = alt_names

In [67]:
for g in data.keys():
    try:
        print(g, reverse_alt_name_dic[g])
    except Exception as e:
        print("Failed:", e)

47Tuc ['NGC104', '47Tuc']
NGC288 ['NGC288']
NGC362 ['NGC362']
Whiting1 ['Whiting1']
NGC1261 ['NGC1261']
Pal1 ['Pal1']
AM1 ['AM1']
Eridanus ['Eridanus']
Pal2 ['Pal2']
NGC1851 ['NGC1851']
M79 ['NGC1904', 'M79']
NGC2298 ['NGC2298']
NGC2419 ['NGC2419']
Ko2 ['Ko2']
Pyxis ['Pyxis']
NGC2808 ['NGC2808']
E3 ['ESO371', 'E3']
Pal3 ['Pal3']
NGC3201 ['NGC3201']
ESO93 ['ESO93']
Pal4 ['Pal4']
Crater ['Laevens1', 'Crater']
Bliss1 ['Bliss1']
Ko1 ['Ko1']
NGC4147 ['NGC4147']
NGC4372 ['NGC4372']
Rup106 ['Rup106']
M68 ['NGC4590', 'M68']
BH140 ['BH140']
NGC4833 ['NGC4833']
M53 ['NGC5024', 'M53']
NGC5053 ['NGC5053']
Kim3 ['Kim3']
oCen ['NGC5139', 'oCen']
M3 ['NGC5272', 'M3']
NGC5286 ['NGC5286']
AM4 ['AM4']
NGC5466 ['NGC5466']
NGC5634 ['NGC5634']
NGC5694 ['NGC5694']
IC4499 ['IC4499']
Munoz1 ['Munoz1']
NGC5824 ['NGC5824']
Pal5 ['Pal5']
NGC5897 ['NGC5897']
M5 ['NGC5904', 'M5']
NGC5927 ['NGC5927']
NGC5946 ['NGC5946']
BH176 ['BH176']
NGC5986 ['NGC5986']
FSR1716 ['FSR1716']
Pal14 ['Arp1', 'Pal14']
Lynga7 ['Lynga7'

## CHECK:
- 2MASS, 2MASSGC02, 2MS-GC01, GLIMPSE2  
- RLGC and RYU  
- SagittariusII  
- ESO280SC06, ESO280  
- NGC7492 has '-' and ''?

In [68]:
print(data["Terzan8"]["Fe_H"])

{'Kruijssen19': -2.18, 'Dotter10': -2.4, 'Forbes10': -1.8, 'VandenBerg13': -2.34, 'Harris10': -2.16, 'Vasquez18': -2.12, 'Saviane12': -2.12, 'Dias15': -2.06, 'Carretta14': -2.27}


In [69]:
# ngcs = ["NGC6528", "NGC6553", "M71", "NGC6558", "NGC6426", "Terzan8"]
ngcs = ["M3", "M15", "M92"]

In [70]:
for gc in ngcs:
    gc = name_parse[gc]
    print(data[gc]["Age"])

{'Kruijssen19': 11.88, 'Dotter10': 12.5, 'Forbes10': 11.39, 'VandenBerg13': 11.75, 'OMalley17': 11.3, 'Vandenberg16': 12.5}
{'Kruijssen19': 12.98, 'Dotter10': 13.25, 'Forbes10': 12.93, 'VandenBerg13': 12.75, 'Roediger13': 12.9, 'OMalley17': 12.6, 'Vandenberg16': 12.5}
{'Kruijssen19': 13.06, 'Dotter10': 13.25, 'Forbes10': 13.18, 'VandenBerg13': 12.75, 'OMalley17': 13.1, 'Vandenberg16': 12.5}


# Combining AMR Data
- Check Prefered status. If none found, then:
- Prefer the most recent data
- If age-metallicity found together, prefer the result
- Average the result?

## List Data

In [71]:
names = list(data.keys())
print(len(names))
N_Met = 0
N_Age = 0
N_Mass = 0
N_Dyn = 0
for n in names:
    gc_data = data[n]
    props = list(gc_data.keys())
    if "Fe_H" in props:
        N_Met += 1
        # print(n, gc_data["Fe_H"])
    if "Age" in props:
        N_Age += 1
    if "Mass" in props:
        N_Mass += 1
    if "vlos" in props:
        N_Dyn += 1
print(f"N Met: {N_Met}")
print(f"N Age: {N_Age}")
print(f"N Mass: {N_Mass}")
print(f"N Dyn: {N_Dyn}")

183
N Met: 171
N Age: 129
N Mass: 166
N Dyn: 176


# Final Data

In [ ]:
def find_latest_data(data):
    names = list(data.keys())
    latest_data = {}
    for gc in names:
        latest_data[gc] = find_latest_individual_data(data[gc])
    return latest_data

def find_latest_individual_data(gc_data):
    props = list(gc_data.keys())
    for x in props:
        papers = list(gc_data[x].keys())
        for p in papers:
            year = paper_age[p]
        
    return

## List Data

## Require Dynamics

## Require Dynamics, AMR, Mass/Mv